In [1]:
from theano.sandbox import cuda

Using cuDNN version 5103 on context None
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 140, in init_dev
    context.free_gmem / MB))
RuntimeError: Trying to preallocate 10867 MB of GPU memory while only 371 MB are available.


SkipTest: You are importing theano.sandbox.cuda. This is the old GPU back-end and is removed from Theano. Use Theano 0.9 to use it. Even better, transition to the new GPU back-end! See https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

In [ ]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [ ]:
path = "data/fisheries/"
batch_size=64

In [ ]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

(val_classes, trn_classes, val_labels, trn_labels,
    val_filenames, filenames, test_filenames) = get_classes(path)

In [ ]:
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

# Basic VGG

In [ ]:
from vgg16bn import Vgg16BN
model = vgg_ft_bn(8)

In [ ]:
trn = get_data(path+'train')
val = get_data(path+'valid')

In [ ]:
test = get_data(path+'test')

In [ ]:
save_array(path+'results/trn.dat', trn)
save_array(path+'results/val.dat', val)

In [ ]:
save_array(path+'results/test.dat', test)

In [ ]:
trn = load_array(path+'results/trn.dat')
val = load_array(path+'results/val.dat')

In [ ]:
test = load_array(path+'results/test.dat')

In [ ]:
gen = image.ImageDataGenerator()

In [ ]:
model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(trn, trn_labels, batch_size=batch_size, nb_epoch=3, validation_data=(val, val_labels))

In [ ]:
model.save_weights(path+'results/ft1.h5')

### Precompute convolutional output

In [ ]:
model.load_weights(path+'results/ft1.h5')

In [ ]:
conv_layers, fc_layrs = split_at(model, Convolution2D)

In [ ]:
conv_model = Sequential(conv_layers)

In [ ]:
conv_feat = conv_model.predict(trn)
conv_val_feat = conv_model.predict(val)

In [ ]:
conv_test_feat = conv_model.predict(test)

In [ ]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [ ]:
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [ ]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [ ]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [ ]:
conv_val_feat.shape

### Train Model

In [ ]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p/4),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(8, activation='softmax')
    ]

In [ ]:
p=0.6

In [ ]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=3,
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=1e-4

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=7,
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.save_weights(path+'models/conv_512_6.h5')

In [ ]:
bn_model.evaluate(conv_val_feat, val_labels)

In [ ]:
bn_model.load_weights(path+'models/conv_512_6.h5')

### Multi-Input

In [ ]:
sizes = [PIL.Image.open(path+'train/'+f).size for f in filenames]
id2size = list(set(sizes))
size2id = {o:i for i,o in enumerate(id2size)}

In [ ]:
import collections
collections.Counter(sizes)

In [ ]:
trn_sizes_orig = to_categorical([size2id[o] for o in sizes], len(id2size))

In [ ]:
raw_val_sizes = [PIL.Image.open(path+'valid/'+f).size for f in val_filenames]
val_sizes = to_categorical([size2id[o] for o in raw_val_sizes], len(id2size))

In [ ]:
trn_sizes = trn_sizes_orig-trn_sizes.orig.mean(axis=0)/trn_sizes_orig.std(axis=0)
val_sizes = val_sizes-trn_sizes_orig.mean(axis=0)/trn_sizes_orig.std(axis=0)

In [ ]:
p=0.6

In [ ]:
inp = Input(conv_layers[-1].output_shape[1:])
sz_inp = Input((len(id2size),))
bn_inp = BatchNormalization()(sz_inp)

x = MaxPooling2D()(inp)
x = BatchNormalization(axis=1)(x)
x = Dropout(p/4)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization(x)
x = Dropout(p/2)(x)
x = merge([x, bn_inp], 'concat')
x = Dense(8, activation='softmax')(x)

In [ ]:
model = Model([inp, sz_inp], x)
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit([conv_fat, trn_sizes], trn_labels, batch_size=batch_size, nb_epoch=3,
          validation_data=([conv_val_feat, val_sizes], val_labels))

In [ ]:
bn_model.optimizer.lr=1e-4

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=8,
            validation_data=(conv_val_feat, val_labels))

# Bounding boxes and Multi output

## Import / view bounding boxes

In [ ]:
import ujson as json

In [ ]:
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft']

In [ ]:
def get_annotations():
    annot_urls = {
        '5458/bet_labels.json': 'bd20591439b650f44b36b72a98d3ce27',
        '5459/shark_labels.json': '94b1b3110ca58ff4788fb659eda7da90',
        '5460/dol_labels.json': '91a25d29a29b7e8b8d7a8770355993de',
        '5461/yft_labels.json': '9ef63caad8f076457d48a21986d81ddc',
        '5462/alb_labels.json': '731c74d347748b5272042f0661dad37c',
        '5463/lag_labels.json': '92d75d9218c3333ac31d74125f2b380a'
    }
    cache_subdir = os.path.abspath(os.path.join(path, 'annos'))
    url_prefix = 'https://kaggle2.blob.core.windows.net/forum-message-attachments/147157/'
    
    if not os.path.exists(cache_subdir):
        os.makedirs(cache_subdir)
    
    for url_suffix, md5_hash in annot_urls.iteritems():
        fname = url_suffix.rsplit('/', 1)[-1]
        get_file(fname, url_prefix + url_suffix, cache_subdir=cache_subdir, md5_hash=md5_hash)

In [ ]:
get_annotations()

In [ ]:
bb_json = {}
for c in anno_classes:
    if c == 'other': continue # no annotation file for "other" class
    j = json.load(open('{}annos/{}_labels.json'.format(path, c), 'r'))
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations'])>0:
            bb_json[l['filename'].split('/')[-1]] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]

In [ ]:
bb_json['img_04908.jpg']

In [ ]:
file2idx = {o:i for i,o in enumerate(raw_filenames)}
val_file2idx = {o:i for i,o in enumerate(raw_val_filenames)}

In [ ]:
empty_bbox = {'height': 0., 'width': 0., 'x': 0., 'y': 0.}

In [ ]:
for f in raw_filenames:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox
for f in raw_val_filenames:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox

In [ ]:
bb_params = ['height', 'width', 'x', 'y']
def convert_bb(bb, size):
    bb = [bb[p] for p in bb_params]
    conv_x = (224. / size[0])
    conv_y = (224. / size[1])
    bb[0] = bb[0]*conv_y
    bb[1] = bb[1]*conv_x
    bb[2] = max(bb[2]*conv_x, 0)
    bb[3] = max(bb[3]*conv_y, 0)
    return bb

In [ ]:
trn_bbox = np.stack([convert_bb(bb_json[f], s) for f,s in zip(raw_filenames, sizes)], 
                   ).astype(np.float32)
val_bbox = np.stack([convert_bb(bb_json[f], s) 
                   for f,s in zip(raw_val_filenames, raw_val_sizes)]).astype(np.float32)

In [ ]:
def create_rect(bb, color='red'):
    return plt.Rectangle((bb[2], bb[3]), bb[1], bb[0], color=color, fill=False, lw=3)

def show_bb(i):
    bb = val_bbox[i]
    plot(val[i])
    plt.gca().add_patch(create_rect(bb))

In [ ]:
show_bb(0)

## Create & train model

In [ ]:
p=0.6

In [ ]:
inp = Input(conv_layers[-1].output_shape[1:])
x = MaxPooling2D()(inp)
x = BatchNormalization(axis=1)(x)
x = Dropout(p/4)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p/2)(x)
x_bb = Dense(4, name='bb')(x)
x_class = Dense(8, activation='softmax', name='class')(x)

In [ ]:
model = Model([inp], [x_bb, x_class])
model.compile(Adam(lr=0.001), loss=['mse', 'categorical_crossentropy'], metrics=['accuracy'],
             loss_weights=[.001, 1.])

In [ ]:
model.fit(conv_feat, [trn_bbox, trn_labels], batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, [val_bbox, val_labels]))

In [ ]:
model.optimizer.lr = 1e-5

In [ ]:
model.fit(conv_feat, [trn_bbox, trn_labels], batch_size=batch_size, nb_epoch=10, 
             validation_data=(conv_val_feat, [val_bbox, val_labels]))

In [ ]:
pred = model.predict(conv_val_feat[0:10])

In [ ]:
def show_bb_pred(i):
    bb = val_bbox[i]
    bb_pred = pred[0][i]
    plt.figure(figsize=(6,6))
    plot(val[i])
    ax=plt.gca()
    ax.add_patch(create_rect(bb_pred, 'yellow'))
    ax.add_patch(create_rect(bb))

In [ ]:
show_bb_pred(6)

In [ ]:
model.evaluate(conv_val_feat, [val_bbox, val_labels])

In [ ]:
model.save_weights(path+'models/bn_anno.h5')

In [ ]:
model.load_weights(path+'models/bn_anno.h5')

## Larger size

### Set up data

In [ ]:
trn = get_data(path+'train', (360, 640))
val = get_data(path+'valid', (360, 640))

In [ ]:
plot(trn[0])

In [ ]:
test = get_data(path+'test', (360, 640))

In [ ]:
save_array(path+'results/trn_640.dat', trn)
save_array(path+'results/val_640.dat', val)

In [ ]:
save_array(path+'results/test_640.dat', test)

In [ ]:
trn = load_array(path+'results/trn_640.dat')
val = load_array(path+'results/val_640.dat')

In [ ]:
vgg640 = Vgg16BN((360,640)).model
vgg640.pop()
vgg640.input_shape, vgg640.output_shape
vgg640.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
conv_val_feat = vgg640.predict(val, batch_size=32, verbose=1)
conv_trn_feat = vgg640.predict(trn, batch_size=32, verbose=1)

In [ ]:
save_array(path+'results/conv_val_640.dat', conv_val_feat)
save_array(path+'results/conv_trn_640.dat', conv_trn_feat)

In [ ]:
conv_test_feat = vgg640.predict(test, batch_size=32, verbose=1)

In [ ]:
save_array(path+'results/conv_test_640.dat', conv_test_feat)

In [ ]:
conv_val_feat = load_array(path+'results/conv_val_640.dat')
conv_trn_feat = load_array(path+'results/conv_trn_640.dat')

In [ ]:
conv_test_feat = load_array(path+'results/conv_test_640.dat')

### Fully Convolutional Net

In [ ]:
conv_layers,_ = split_at(vgg640, Convolution2D)

In [ ]:
nf = 120; p=0.

In [ ]:
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape = conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(8,3,3, activation='relu', border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

In [ ]:
lrg_model = Sequential(get_lrg_layers())

In [ ]:
lrg_model.summary

In [ ]:
lrg_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
lrg_model.fit(conv_trn_ft, trn_labels, batch_size=batch_size, nb_epoch=2,
             validation_data=(conv_val_ft, val_labels))

In [ ]:
lrg_model.optimizer.lr = 1e-5

In [ ]:
lrg_model.fit(conv_trn_ft, trn_labels, batch_size=batch_size, nb_epoch=6,
             validation_data=(conv_val_ft, val_labels))

In [ ]:
lrg_model.save_weights(path+'models/lrg_nmp.h5')

In [ ]:
lrg_model.load_weights(path+'models/lrg_nmp.h5')

In [ ]:
lrg_model.evaluate(conv_val_feat, val_labels)

In [ ]:
#grab output of last conv layer in lrg_model, the 4th to last:
l = lrg_model.layers
conv_fn = K.function([1[0].input, K.learning_phase()], l[-4].output)

In [ ]:
#TODO: make sure you understand what this is doing
def get_cm(inp, label):
    conv = conv_fn([inp,0])[0,label]
    return scipy.misc.imresize(conv, (360,640), interp='nearest')

In [ ]:
#we add an extra dimension to input since CNN expects a batch
inp = np.expand_dims(conv_val_feat[0], 0)
np.round(lrg_model.predict(inp)[0], 2)

In [ ]:
plt.imshow(to_plot(val[0]))

In [ ]:
cm = get_cm(inp,0)

In [ ]:
plt.imshow(cm, cmap="hot")

### All Convolutional Net Heatmap

In [ ]:
# same lrg_layers as before, w/ MaxPooling removed
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        Convolution2D(8,3,3, border_mode='same'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

In [ ]:
lrg_model = Sequential(get_lrg_layers())

In [ ]:
lrg_model.summary()

In [ ]:
lrg_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
lrg_model.fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=2,
             validation_data=(conv_val_feat, val_labels))

In [ ]:
lrg_model.optimizer.lr=1e-5

In [ ]:
lrg_model.fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=6,
             validation_data=(conv_val_feat, val_labels))

In [ ]:
lrg_model.save_weights(path+'models/lrg_0mp.h5')

In [ ]:
lrg_model.load_weights(path+'models/lrg_0mp.h5')

#### Create Heatmap

In [ ]:
l = lrg_model.layers
conv_fn = K.function([l[0].input, K.learning_phase], l[-3].output)

In [ ]:
def get_cm2(inp, label):
    conv = conv_fn([inp,0])[0,label]
    return scipy.misc.imresize(conv, (360,640))

In [ ]:
inp = np.expand_dimsm(conv_val_feat[0],0)

In [ ]:
plt.imshow(to_plot(val[0]))

In [ ]:
cm = get_cm2(inp,0)

In [ ]:
cm = get_cm2(inp, 4)

In [ ]:
plt.imshow(cm, cmap="hot")

In [ ]:
plt.figure(figsize=(10,10))
plot(val[0])
plt.imshow(cm, cmap="hot", alpha=0.5)

### Inception mini-net

In [ ]:
def conv2d_bn(x, nb_filter, nb_row, nb_col, subsample=(1,1)):
    x = Convolution2D(nb_filter, nb_row, nb_col,
                     subsample=subsample, activation='relu', border_mode='same')(x)
    return BatchNormalization(axis=1)(x)

In [ ]:
def incep_block(x):
    branch1x1 = conv2d_bn(x, 32, 1, 1, subsample=(2,2))
    branch5x5 = conv2d_bn(x, 24, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 32, 5, 5, subsample=(2,2))
    
    branch3x3dbl = conv2d_bn(x, 32, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 48, 3, 3)
    branch3x3dbl = conv2d_bn(branch_3x3dbl, 48, 3, 3, subsample=(2,2)) #what is subsample?
    
    branch_pool = AveragePooling2D(
        (3,3), strides=(2,2), border_mode='same')(x)
    branch_pool = conv2d_bn(branch_pool, 16, 1, 1)
    return merge([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                mode='concat', concat_axis=1)

In [ ]:
inp = Input(vgg640.layers[-1].output_shape[1:])
x = BatchNormalization(axis=1)(inp)
x = incep_block(x)
x = incep_block(x)
x = incep_block(x)
x = Dropout(0.75)(x)
x = Convolution2D(8,3,3, border_mode='same')(x)
x = GlobalAveragePooling2D()(x)
outp = Activation('softmax')(x)

In [ ]:
lrg_model = Model([inp], outp)